# 20. JSONデータの読み込み

In [2]:
#20 Wikipedia記事のJSONファイルを読み込み，「イギリス」に関する記事本文を表示せよ．問題21-29では，ここで抽出した記事本文に対して実行せよ．

import gzip
import json

with gzip.open('./data/jawiki-country.json.gz') as raw, open('./work/wiki_uk.txt', 'w') as out:
    for line in raw:
        data = json.loads(line)
        if data['title'] == 'イギリス':
            print(data['text'][:1000]) #長すぎるから1000文字だけ
            
            # 追記: 外部ファイルに保存するようにした
            print(data['text'], file = out, end = '')

{{redirect|UK}}
{{基礎情報 国
|略名 = イギリス
|日本語国名 = グレートブリテン及び北アイルランド連合王国
|公式国名 = {{lang|en|United Kingdom of Great Britain and Northern Ireland}}<ref>英語以外での正式国名:<br/>
*{{lang|gd|An Rìoghachd Aonaichte na Breatainn Mhòr agus Eirinn mu Thuath}}（[[スコットランド・ゲール語]]）<br/>
*{{lang|cy|Teyrnas Gyfunol Prydain Fawr a Gogledd Iwerddon}}（[[ウェールズ語]]）<br/>
*{{lang|ga|Ríocht Aontaithe na Breataine Móire agus Tuaisceart na hÉireann}}（[[アイルランド語]]）<br/>
*{{lang|kw|An Rywvaneth Unys a Vreten Veur hag Iwerdhon Glédh}}（[[コーンウォール語]]）<br/>
*{{lang|sco|Unitit Kinrick o Great Breetain an Northren Ireland}}（[[スコットランド語]]）<br/>
**{{lang|sco|Claught Kängrick o Docht Brätain an Norlin Airlann}}、{{lang|sco|Unitet Kängdom o Great Brittain an Norlin Airlann}}（アルスター・スコットランド語）</ref>
|国旗画像 = Flag of the United Kingdom.svg
|国章画像 = [[ファイル:Royal Coat of Arms of the United Kingdom.svg|85px|イギリスの国章]]
|国章リンク = （[[イギリスの国章|国章]]）
|標語 = {{lang|fr|Dieu et mon droit}}<br/>（[[フランス語]]:神と私の権利）
|国歌 = [[女王陛下万歳|神よ女王陛下を守り給え]]
|位置画像 = Location_UK_

!wget -NP (dir) (url)で保存できる
-P:保存するディレクトリを指定するオプション -N:リンク先のデータが今あるデータよりも新しい場合のみDLする

!zcat (fname)
gzipで圧縮されたファイルをそのままシェル上で見られる

# 21. カテゴリ名を含む行を抽出

In [16]:
#21 記事中でカテゴリ名を宣言している行を抽出せよ．

import gzip
import json
import re

with gzip.open('./data/jawiki-country.json.gz') as raw:
    for line in raw:
        data = json.loads(line)
        if data['title'] == 'イギリス':
            body = data['text']
            
ptn_category = re.compile(r'^\[\[Category:.*\]\]$', flags = re.MULTILINE)

for cat in ptn_category.finditer(body):
    print(cat.group()) # match.group → matchオブジェクトに対してパターンにマッチする部分全体を返す

[[Category:イギリス|*]]
[[Category:英連邦王国|*]]
[[Category:G8加盟国]]
[[Category:欧州連合加盟国]]
[[Category:海洋国家]]
[[Category:君主国]]
[[Category:島国|くれいとふりてん]]
[[Category:1801年に設立された州・地域]]


括弧は左側でエスケープしていれば右側は自動的にエスケープしたものと見なされるからバックスラッシュは左だけでよい

# 22. カテゴリ名の抽出

In [15]:
#22 記事のカテゴリ名を（行単位ではなく名前で）抽出せよ．

import re
import json

ptn_category = re.compile(r'^\[\[Category:(.*?)(?:\|.*)?\]\]$')

with open('./work/wiki_uk.txt') as data:
    for line in data:
        # searchはmatchオブジェクトを返す
        match = ptn_category.search(line)
        if match:
            print(match.group(1))

イギリス
英連邦王国
G8加盟国
欧州連合加盟国
海洋国家
君主国
島国
1801年に設立された州・地域


# 23. セクション構造

In [14]:
#23 記事中に含まれるセクション名とそのレベル（例えば"== セクション名 =="なら1）を表示せよ．

import re
import json

ptn_section = re.compile(r'^(={2,})(?:\s)?(.*?)(?:\s)?\1$')

with open('./work/wiki_uk.txt') as data:
    for line in data:
        match = ptn_section.search(line)
        if match:
            sect_level = match.end(1) - match.start(1) - 1
            print(match.group(2), sect_level)

国名 1
歴史 1
地理 1
気候 2
政治 1
外交と軍事 1
地方行政区分 1
主要都市 2
科学技術 1
経済 1
鉱業 2
農業 2
貿易 2
通貨 2
企業 2
交通 1
道路 2
鉄道 2
海運 2
航空 2
通信 1
国民 1
言語 2
宗教 2
婚姻 2
教育 2
文化 1
食文化 2
文学 2
哲学 2
音楽 2
イギリスのポピュラー音楽 3
映画 2
コメディ 2
国花 2
世界遺産 2
祝祭日 2
スポーツ 1
サッカー 2
競馬 2
モータースポーツ 2
脚注 1
関連項目 1
外部リンク 1


# 24. ファイル参照の抽出

In [17]:
#24 記事から参照されているメディアファイルをすべて抜き出せ．

import re
import json

ptn_fileref = re.compile(r'(?:ファイル|File):(?P<fname>.*?)\|') # ?Pでグループに名前付けできる

with open('./work/wiki_uk.txt') as data:
    for line in data:
        match = ptn_fileref.search(line)
        if match:
            print(match.group('fname'))

Royal Coat of Arms of the United Kingdom.svg
Battle of Waterloo 1815.PNG
The British Empire.png
Uk topo en.jpg
BenNevis2005.jpg
Elizabeth II greets NASA GSFC employees, May 8, 2007 edit.jpg
Palace of Westminster, London - Feb 2007.jpg
David Cameron and Barack Obama at the G20 Summit in Toronto.jpg
Soldiers Trooping the Colour, 16th June 2007.jpg
Scotland Parliament Holyrood.jpg
London.bankofengland.arp.jpg
City of London skyline from London City Hall - Oct 2008.jpg
Oil platform in the North SeaPros.jpg
Eurostar at St Pancras Jan 2008.jpg
Heathrow T5.jpg
Anglospeak.svg
CHANDOS3.jpg
The Fabs.JPG
PalaceOfWestminsterAtNight.jpg
Westminster Abbey - West Door.jpg
Edinburgh Cockburn St dsc06789.jpg
Canterbury Cathedral - Portal Nave Cross-spire.jpeg
Kew Gardens Palm House, London - July 2009.jpg
2005-06-27 - United Kingdom - England - London - Greenwich.jpg
Stonehenge2007 07 30.jpg
Yard2.jpg
Durham Kathedrale Nahaufnahme.jpg
Roman Baths in Bath Spa, England - July 2006.jpg
Fountains Abbey vie

# 25. テンプレートの抽出

In [19]:
#25 記事中に含まれる「基礎情報」テンプレートのフィールド名と値を抽出し，辞書オブジェクトとして格納せよ．

import re
import json

# 基礎情報抜き出し用の正規表現パターン
ptn_info = re.compile(r'^\{\{基礎情報(.*?)^\}\}$', flags = re.MULTILINE | re.DOTALL) # 改行を.にマッチ
# 辞書に格納するためのfield/value分割
ptn_dict = re.compile(r'^\|(?P<field>.*?)(\s)+(=)(\s)+(?P<value>.*?)(?=\n^\||\n^$)', flags = re.MULTILINE | re.DOTALL)

with open('./work/wiki_uk.txt') as data:
    dict_info = {}
    
    # 基礎情報部分を拾うためにfile.read()してるのはmemory unfriendly...
    for info in ptn_dict.finditer(ptn_info.search(data.read()).group(1)):
        dict_info[info.group('field')] = info.group('value')

    for key, val in dict_info.items():
        print(key, ':', val)

略名 : イギリス
日本語国名 : グレートブリテン及び北アイルランド連合王国
公式国名 : {{lang|en|United Kingdom of Great Britain and Northern Ireland}}<ref>英語以外での正式国名:<br/>
*{{lang|gd|An Rìoghachd Aonaichte na Breatainn Mhòr agus Eirinn mu Thuath}}（[[スコットランド・ゲール語]]）<br/>
*{{lang|cy|Teyrnas Gyfunol Prydain Fawr a Gogledd Iwerddon}}（[[ウェールズ語]]）<br/>
*{{lang|ga|Ríocht Aontaithe na Breataine Móire agus Tuaisceart na hÉireann}}（[[アイルランド語]]）<br/>
*{{lang|kw|An Rywvaneth Unys a Vreten Veur hag Iwerdhon Glédh}}（[[コーンウォール語]]）<br/>
*{{lang|sco|Unitit Kinrick o Great Breetain an Northren Ireland}}（[[スコットランド語]]）<br/>
**{{lang|sco|Claught Kängrick o Docht Brätain an Norlin Airlann}}、{{lang|sco|Unitet Kängdom o Great Brittain an Norlin Airlann}}（アルスター・スコットランド語）</ref>
国旗画像 : Flag of the United Kingdom.svg
国章画像 : [[ファイル:Royal Coat of Arms of the United Kingdom.svg|85px|イギリスの国章]]
国章リンク : （[[イギリスの国章|国章]]）
標語 : {{lang|fr|Dieu et mon droit}}<br/>（[[フランス語]]:神と私の権利）
国歌 : [[女王陛下万歳|神よ女王陛下を守り給え]]
位置画像 : Location_UK_EU_Europe_001.svg
公用語 : [[英語]]（事実上

In [1]:
# itertools.groupbyの使い方
from itertools import groupby
num = [1, 2, 4, 3, 5, 8, 10]
for key, group in groupby(num, key = lambda n: n % 2 == 0):
    #groupby(iterable, key)
    print(key, list(group))

False [1]
True [2, 4]
False [3, 5]
True [8, 10]


# 26. 強調マークアップの除去

In [20]:
#26 25の処理時に，テンプレートの値からMediaWikiの強調マークアップ（弱い強調，強調，強い強調のすべて）を除去してテキストに変換せよ（参考: マークアップ早見表）．

import re
import json

# 基礎情報抜き出し用の正規表現パターン
ptn_info = re.compile(r'^\{\{基礎情報(.*?)^\}\}$', flags = re.MULTILINE | re.DOTALL) # 改行を.にマッチ
# 辞書に格納するためのfield/value分割
ptn_dict = re.compile(r'^\|(?P<field>.*?)(\s)+(=)(\s)+(?P<value>.*?)(?=\n^\||\n^$)', flags = re.MULTILINE | re.DOTALL)

ptn_mkup = re.compile(r'(\'{2,})(.*?)\1', flags = re.MULTILINE)

with open('./work/wiki_uk.txt') as data:
    dict_info = {}

    for info in ptn_dict.finditer(ptn_info.search(data.read()).group(1)):
        # sub()は破壊的ではない → 代入する必要
        dict_info[info.group('field')] = ptn_mkup.sub(r'\2', info.group('value'))

    for key, val in dict_info.items():
        print(key, ':', val)

略名 : イギリス
日本語国名 : グレートブリテン及び北アイルランド連合王国
公式国名 : {{lang|en|United Kingdom of Great Britain and Northern Ireland}}<ref>英語以外での正式国名:<br/>
*{{lang|gd|An Rìoghachd Aonaichte na Breatainn Mhòr agus Eirinn mu Thuath}}（[[スコットランド・ゲール語]]）<br/>
*{{lang|cy|Teyrnas Gyfunol Prydain Fawr a Gogledd Iwerddon}}（[[ウェールズ語]]）<br/>
*{{lang|ga|Ríocht Aontaithe na Breataine Móire agus Tuaisceart na hÉireann}}（[[アイルランド語]]）<br/>
*{{lang|kw|An Rywvaneth Unys a Vreten Veur hag Iwerdhon Glédh}}（[[コーンウォール語]]）<br/>
*{{lang|sco|Unitit Kinrick o Great Breetain an Northren Ireland}}（[[スコットランド語]]）<br/>
**{{lang|sco|Claught Kängrick o Docht Brätain an Norlin Airlann}}、{{lang|sco|Unitet Kängdom o Great Brittain an Norlin Airlann}}（アルスター・スコットランド語）</ref>
国旗画像 : Flag of the United Kingdom.svg
国章画像 : [[ファイル:Royal Coat of Arms of the United Kingdom.svg|85px|イギリスの国章]]
国章リンク : （[[イギリスの国章|国章]]）
標語 : {{lang|fr|Dieu et mon droit}}<br/>（[[フランス語]]:神と私の権利）
国歌 : [[女王陛下万歳|神よ女王陛下を守り給え]]
位置画像 : Location_UK_EU_Europe_001.svg
公用語 : [[英語]]（事実上

# 27. 内部リンクの除去

In [21]:
#27 26の処理に加えて，テンプレートの値からMediaWikiの内部リンクマークアップを除去し，テキストに変換せよ（参考: マークアップ早見表）．

import re
import json

# 基礎情報抜き出し用の正規表現パターン
ptn_info = re.compile(r'^\{\{基礎情報(.*?)^\}\}$', flags = re.MULTILINE | re.DOTALL) # 改行を.にマッチ
# 辞書に格納するためのfield/value分割
ptn_dict = re.compile(r'^\|(?P<field>.*?)(\s)+(=)(\s)+(?P<value>.*?)(?=\n^\||\n^$)', flags = re.MULTILINE | re.DOTALL)

ptn_mkup = re.compile(r'(\'{2,})(.*?)\1', flags = re.MULTILINE)

ptn_intlink = re.compile(r'\[\[(?!ファイル:|Category:)(?:[^\[\]]*?\|)?([^\[\]]*?)\]\]', flags = re.MULTILINE) # 否定先読み (両側かっこを除いた貪欲マッチ)

with open('./work/wiki_uk.txt') as data:
    dict_info = {}
    for info in ptn_dict.finditer(ptn_info.search(data.read()).group(1)):
        mkup_removed = ptn_mkup.sub(r'\2', info.group('value'))
        dict_info[info.group('field')] = ptn_intlink.sub(r'\1', mkup_removed)

    for key, val in dict_info.items():
        print(key, ':', val)

略名 : イギリス
日本語国名 : グレートブリテン及び北アイルランド連合王国
公式国名 : {{lang|en|United Kingdom of Great Britain and Northern Ireland}}<ref>英語以外での正式国名:<br/>
*{{lang|gd|An Rìoghachd Aonaichte na Breatainn Mhòr agus Eirinn mu Thuath}}（スコットランド・ゲール語）<br/>
*{{lang|cy|Teyrnas Gyfunol Prydain Fawr a Gogledd Iwerddon}}（ウェールズ語）<br/>
*{{lang|ga|Ríocht Aontaithe na Breataine Móire agus Tuaisceart na hÉireann}}（アイルランド語）<br/>
*{{lang|kw|An Rywvaneth Unys a Vreten Veur hag Iwerdhon Glédh}}（コーンウォール語）<br/>
*{{lang|sco|Unitit Kinrick o Great Breetain an Northren Ireland}}（スコットランド語）<br/>
**{{lang|sco|Claught Kängrick o Docht Brätain an Norlin Airlann}}、{{lang|sco|Unitet Kängdom o Great Brittain an Norlin Airlann}}（アルスター・スコットランド語）</ref>
国旗画像 : Flag of the United Kingdom.svg
国章画像 : [[ファイル:Royal Coat of Arms of the United Kingdom.svg|85px|イギリスの国章]]
国章リンク : （国章）
標語 : {{lang|fr|Dieu et mon droit}}<br/>（フランス語:神と私の権利）
国歌 : 神よ女王陛下を守り給え
位置画像 : Location_UK_EU_Europe_001.svg
公用語 : 英語（事実上）
首都 : ロンドン
最大都市 : ロンドン
元首等肩書 : 女王
元首等氏名 : エリザベス2世


re.sub()はregexパターンの代わりに関数も引数にとれる

# 28. MediaWikiマークアップの除去

In [17]:
# 27の処理に加えて，テンプレートの値からMediaWikiマークアップを可能な限り除去し，国の基本情報を整形せよ．

import re
import json


# 基礎情報抜き出し用の正規表現パターン
ptn_info = re.compile(r'^\{\{基礎情報(.*?)^\}\}$', flags = re.MULTILINE | re.DOTALL) # 改行を.にマッチ
# 辞書に格納するためのfield/value分割
ptn_dict = re.compile(r'^\|(?P<field>.*?)(\s)+(=)(\s)+(?P<value>.*?)(?=\n^\||\n^$)', flags = re.MULTILINE | re.DOTALL)

ptn_mkup = re.compile(r'(\'{2,})(.*?)\1', flags = re.MULTILINE)

ptn_intlink = re.compile(r'\[\[(?:[^\[\]]*?\|){0,2}([^\[\]]*?)\]\]', flags = re.MULTILINE) # ファイルにもマッチするように変更
ptn_extlink = re.compile(r'\[?http(.*?\s)?(.*?)\]?', flags = re.MULTILINE)

ptn_lang = re.compile(r'\{\{lang\|.*?\|(.*?)\}\}', flags = re.MULTILINE)

ptn_tag = re.compile(r'<.*?(?:ref|br).*?>', flags = re.MULTILINE)

with open('./work/wiki_uk.txt') as data:
    dict_info = {}
    for info in ptn_dict.finditer(ptn_info.search(data.read()).group(1)):
        
        # 順を追って除去していく
        mkup_removed = ptn_mkup.sub(r'\2', info.group('value'))
        link_removed = ptn_extlink.sub(r'\2', ptn_intlink.sub(r'\1', mkup_removed))
        lang_removed = ptn_lang.sub(r'\1', link_removed)
        dict_info[info.group('field')] = ptn_tag.sub('', lang_removed)

    for key, val in dict_info.items():
        print(key, ':', val)

略名 : イギリス
日本語国名 : グレートブリテン及び北アイルランド連合王国
公式国名 : United Kingdom of Great Britain and Northern Ireland英語以外での正式国名:
*An Rìoghachd Aonaichte na Breatainn Mhòr agus Eirinn mu Thuath（スコットランド・ゲール語）
*Teyrnas Gyfunol Prydain Fawr a Gogledd Iwerddon（ウェールズ語）
*Ríocht Aontaithe na Breataine Móire agus Tuaisceart na hÉireann（アイルランド語）
*An Rywvaneth Unys a Vreten Veur hag Iwerdhon Glédh（コーンウォール語）
*Unitit Kinrick o Great Breetain an Northren Ireland（スコットランド語）
**Claught Kängrick o Docht Brätain an Norlin Airlann、Unitet Kängdom o Great Brittain an Norlin Airlann（アルスター・スコットランド語）
国旗画像 : Flag of the United Kingdom.svg
国章画像 : イギリスの国章
国章リンク : （国章）
標語 : Dieu et mon droit（フランス語:神と私の権利）
国歌 : 神よ女王陛下を守り給え
位置画像 : Location_UK_EU_Europe_001.svg
公用語 : 英語（事実上）
首都 : ロンドン
最大都市 : ロンドン
元首等肩書 : 女王
元首等氏名 : エリザベス2世
首相等肩書 : 首相
首相等氏名 : デーヴィッド・キャメロン
面積順位 : 76
面積大きさ : 1 E11
面積値 : 244,820
水面積率 : 1.3%
人口統計年 : 2011
人口順位 : 22
人口大きさ : 1 E7
人口値 : 63,181,775United Nations Department of Economic and Social Affairs>Population Division>Data>

複数回のre.sub()正規表現も関数もリスト化すると見通しがよい

# 29. 国旗画像のURLを取得する

requestsオブジェクトに対して
r.url は投げた実際にAPIに送信したURL
r.text はgetリクエストに対してAPIから帰ってきたデータ

In [1]:
#29 テンプレートの内容を利用し，国旗画像のURLを取得せよ．（ヒント: MediaWiki APIのimageinfoを呼び出して，ファイル参照をURLに変換すればよい）

import re
import json
import urllib

# 基礎情報抜き出し用の正規表現パターン
ptn_info = re.compile(r'^\{\{基礎情報(.*?)^\}\}$', flags = re.MULTILINE | re.DOTALL) # 改行を.にマッチ
# 辞書に格納するためのfield/value分割
ptn_dict = re.compile(r'^\|(?P<field>.*?)(\s)+(=)(\s)+(?P<value>.*?)(?=\n^\||\n^$)', flags = re.MULTILINE | re.DOTALL)

ptn_mkup = re.compile(r'(\'{2,})(.*?)\1', flags = re.MULTILINE)

ptn_intlink = re.compile(r'\[\[(?:[^\[\]]*?\|){0,2}([^\[\]]*?)\]\]', flags = re.MULTILINE) # ファイルにもマッチするように変更
ptn_extlink = re.compile(r'\[?http(.*?\s)?(.*?)\]?', flags = re.MULTILINE)

ptn_lang = re.compile(r'\{\{lang\|.*?\|(.*?)\}\}', flags = re.MULTILINE)

ptn_tag = re.compile(r'<.*?(?:ref|br).*?>', flags = re.MULTILINE)

with open('./work/wiki_uk.txt') as data:
    dict_info = {}

    for info in ptn_dict.finditer(ptn_info.search(data.read()).group(1)):

        mkup_removed = ptn_mkup.sub(r'\2', info.group('value'))
        link_removed = ptn_extlink.sub(r'\1', ptn_intlink.sub(r'\1', mkup_removed))
        lang_removed = ptn_lang.sub(r'\1', link_removed)
        dict_info[info.group('field')] = ptn_tag.sub('', lang_removed)
    
    fname = dict_info['国旗画像']
    print(fname)

endpoint = 'https://ja.wikipedia.org/w/api.php?'
param = urllib.parse.urlencode({'format':'json', 'action':'query', 'titles':'file:' + fname, 'prop':'imageinfo', 'iiprop':'url'})
# マッピングオブジェクトをパーセントエンコード(URLで使える文字にエンコード)して&でつないだ文字列を返す
url = endpoint + param

req = urllib.request.Request(url) # HTTPリクエストオブジェクトを作成
with urllib.request.urlopen(req) as res:
    # レスポンスオブジェクトを受け取る
    data = json.loads(res.read().decode('utf-8'))
    print(data['query']['pages']['-1']['imageinfo'][0]['url'])

Flag of the United Kingdom.svg
https://upload.wikimedia.org/wikipedia/commons/a/ae/Flag_of_the_United_Kingdom.svg


![](https://upload.wikimedia.org/wikipedia/commons/a/ae/Flag_of_the_United_Kingdom.svg)

requestsライブラリが便利

In [2]:
import re
import json
import requests

# 基礎情報抜き出し用の正規表現パターン
ptn_info = re.compile(r'^\{\{基礎情報(.*?)^\}\}$', flags = re.MULTILINE | re.DOTALL) # 改行を.にマッチ
# 辞書に格納するためのfield/value分割
ptn_dict = re.compile(r'^\|(?P<field>.*?)(\s)+(=)(\s)+(?P<value>.*?)(?=\n^\||\n^$)', flags = re.MULTILINE | re.DOTALL)

ptn_mkup = re.compile(r'(\'{2,})(.*?)\1', flags = re.MULTILINE)

ptn_intlink = re.compile(r'\[\[(?:[^\[\]]*?\|){0,2}([^\[\]]*?)\]\]', flags = re.MULTILINE) # ファイルにもマッチするように変更
ptn_extlink = re.compile(r'\[?http(.*?\s)?(.*?)\]?', flags = re.MULTILINE)

ptn_lang = re.compile(r'\{\{lang\|.*?\|(.*?)\}\}', flags = re.MULTILINE)

ptn_tag = re.compile(r'<.*?(?:ref|br).*?>', flags = re.MULTILINE)

with open('./work/wiki_uk.txt') as data:
    dict_info = {}

    for info in ptn_dict.finditer(ptn_info.search(data.read()).group(1)):

        mkup_removed = ptn_mkup.sub(r'\2', info.group('value'))
        link_removed = ptn_extlink.sub(r'\1', ptn_intlink.sub(r'\1', mkup_removed))
        lang_removed = ptn_lang.sub(r'\1', link_removed)
        dict_info[info.group('field')] = ptn_tag.sub('', lang_removed)
    
    fname = dict_info['国旗画像']
    print(fname)

query = {'format':'json', 'action':'query', 'titles':'file:' + fname, 'prop':'imageinfo', 'iiprop':'url'}
req = requests.get('https://ja.wikipedia.org/w/api.php?', params = query)

data = req.json()
print(data['query']['pages']['-1']['imageinfo'][0]['url'])

Flag of the United Kingdom.svg
https://upload.wikimedia.org/wikipedia/commons/a/ae/Flag_of_the_United_Kingdom.svg
